In [1]:
import sys
sys.path.insert(1, '/home/amrzr/Work/Codes/Offline_IMOEA_Framework/')

from desdeo_problem.Problem import DataProblem
from main_project_files.surrogate_fullGP import FullGPRegressor as fgp
from main_project_files.surrogate_SVR import SVR
from main_project_files.surrogate_RandomForest import RandomForest
import time
import numpy as np
import pandas as pd

main_directory = 'Pump_Test_Tomas_5_140_all'
data_folder = '/home/amrzr/Work/Codes/data'
#data_file = data_folder+'/pump_data/01_DOE_data.csv'
data_file = data_folder+'/pump_data/02_DOE_140_data.csv'
#data_file = data_folder+'/pump_data/03_DOE_140_all_data.csv'
path = data_folder + '/test_runs/' + main_directory

df = pd.read_csv(data_file)
df[['f1','f2','f3']] = df[['f1','f2','f3']]*-1

x_low = np.ones(22)*0
x_high = np.ones(22)

x_low_new = np.ones(22)*0
x_high_new = np.ones(22)

def scale_data(data):
    x_data = np.asarray(data.loc[:,:'x22'])
    x_data_scaled = (x_data - x_low)/(np.asarray(x_high) - np.asarray(x_low))
    data.loc[:,:'x22'] = x_data_scaled
    return data

def build_surrogates(nobjs, nvars, df): #x_data, y_data):
    x_names = list(df.columns)[0:22]
    y_names = list(df.columns)[22:25]
    row_names = ['lower_bound','upper_bound']
    #data = pd.DataFrame(np.hstack((x_data,y_data)), columns=x_names+y_names)
    #data = df
    #print(data)
    zz = np.asarray(df.loc[:,:'x22'])
    #x_low = np.min(zz,axis=0)
    #x_high = np.max(zz,axis=0)
    #x_low = [20, 0.2, 0.22, 0.25, -5, 85, 355, 450, 15, 15, -10, 16, 0.25, 0.2, 0.25, -5, 85, 450, 15, 15, 27, -15]
    #x_high = [30,0.72, 0.76 , 0.8 , 0 , 90, 380 , 600, 45 , 50,10 , 26, 0.76, 0.7 ,0.76, 0 ,90 ,600 ,60 ,50 ,35 ,5]
    bounds = pd.DataFrame(np.vstack((x_low_new,x_high_new)), columns=x_names, index=row_names)

    problem = DataProblem(data=df, variable_names=x_names, objective_names=y_names,bounds=bounds)
    start = time.time()
    #problem.train(fgp)
    #problem.train(SVR)
    problem.train(RandomForest)
    #problem.train(GaussianProcessRegressor) #, {"kernel": Matern(nu=3/2)})
    end = time.time()
    time_taken = end - start
    return problem

data_scaled = scale_data(df)

In [2]:
nobjs = 3 
nvars = 22
surrogate_problem = build_surrogates(nobjs, nvars, data_scaled)

In [3]:
df

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x16,x17,x18,x19,x20,x21,x22,f1,f2,f3
0,0.013889,0.458333,0.547222,0.991667,0.452778,0.541667,0.386111,0.225000,0.475000,0.958333,...,0.619444,0.986111,0.513889,0.969444,0.941667,0.980556,0.458333,-77.431604,-86.026921,-87.129884
1,0.019444,0.075000,0.291667,0.575000,0.702778,0.063889,0.802778,0.325000,0.091667,0.641667,...,0.347222,0.736111,0.913889,0.719444,0.869444,0.719444,0.713889,-85.291008,-87.037706,-83.393283
2,0.030556,0.941667,0.986111,0.758333,0.175000,0.230556,0.541667,0.752778,0.341667,0.425000,...,0.697222,0.519444,0.252778,0.119444,0.569444,0.547222,0.091667,-79.637733,-87.316725,-88.613472
3,0.047222,0.380556,0.697222,0.847222,0.069444,0.019444,0.147222,0.586111,0.108333,0.563889,...,0.008333,0.286111,0.363889,0.247222,0.458333,0.575000,0.608333,-81.724272,-86.381360,-83.321240
4,0.052778,0.725000,0.730556,0.597222,0.125000,0.713889,0.230556,0.097222,0.919444,0.130556,...,0.169444,0.802778,0.052778,0.102778,0.519444,0.763889,0.002778,-78.052977,-84.862114,-86.566185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,0.969444,0.058333,0.013889,0.241667,0.825000,0.769444,0.458333,0.247222,0.658333,0.575000,...,0.302778,0.480556,0.747222,0.880556,0.430556,0.452778,0.908333,-80.081161,-84.549290,-82.887886
136,0.975000,0.197222,0.936111,0.936111,0.341667,0.252778,0.369444,0.191667,0.247222,0.947222,...,0.091667,0.458333,0.397222,0.780556,0.347222,0.430556,0.836111,-78.521420,-77.997776,-77.637262
137,0.980556,0.925000,0.708333,0.425000,0.297222,0.936111,0.197222,0.675000,0.908333,0.358333,...,0.652778,0.263889,0.086111,0.280556,0.130556,0.280556,0.286111,-78.159703,-83.620652,-87.128623
138,0.991667,0.697222,0.191667,0.430556,0.808333,0.180556,0.708333,0.258333,0.880556,0.547222,...,0.208333,0.275000,0.952778,0.519444,0.891667,0.191667,0.769444,-79.472595,-83.810159,-83.730185


In [10]:
X=df.values[:,0:22]
surrogate_problem.evaluate(X,use_surrogate=True)

ValueError: Expected 2D array, got 1D array instead:
array=[0.01388889 0.45833334 0.5472222  0.9916667  0.45277777 0.5416667
 0.3861111  0.225      0.475      0.9583333  0.5861111  0.91944444
 0.10277778 0.9638889  0.7972222  0.61944443 0.9861111  0.5138889
 0.96944445 0.94166666 0.98055553 0.45833334].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.